In [2]:
!pip install tensorflow keras numpy matplotlib pandas sklearn Bio torch 

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

iris = load_iris()
X = preprocessing.scale(iris['data'])
Y = iris['target']

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from keras.optimizers import Adam, SGD
import math
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from keras.utils import to_categorical
from sklearn.metrics import classification_report 

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
accu_lst = []
sen_lst=[]
pre_lst=[]
acc_lst=[]
mcc_lst=[]
auc_lst=[]

with tf.Session() as sess:

  #training data and test data
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
  x=X_train
  y1=np.array(y_train)
  
  for train, test in kfold.split(x, y1):

      y = to_categorical(y1)
      yy = to_categorical(y_test)

      #model
      model = Sequential()
      model.add(Dense(6, input_dim=4, activation='relu'))
      model.add(Dense(3, activation='softmax'))
      #Compile the model
      model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      
      # Fit the model
      model.fit(x[train], y[train], validation_split=0.10, epochs=12, verbose=0,batch_size=10)

      # evaluate the model
      predictions = model.predict(x[test])
      y_pred=( np.argmax(predictions, axis=1))
      #confution matrix generation
      cm=np.array(confusion_matrix(y1[test],y_pred,labels=[0,1]))
      confusion=pd.DataFrame(cm,index=['not_Glutarylation','Glutarylation'],
                              columns=['predicted_not_Glutarylation','predicted_Glutarylation'])
      
      True_Positive=confusion.at['Glutarylation','predicted_Glutarylation']
      False_Positive=confusion.at['not_Glutarylation','predicted_Glutarylation']
      True_Negative=confusion.at['not_Glutarylation','predicted_not_Glutarylation']
      False_Negative=confusion.at['Glutarylation','predicted_not_Glutarylation']
      
      #Accuracy==>>accuracy = (correctly predicted class / total testing class) × 100%
      # accuracy_score(y_predict, y_test)

      total_testing_class=True_Positive+True_Negative+False_Positive+False_Negative
      correctly_predicted_class=True_Positive + True_Negative
      accuracy = (correctly_predicted_class / total_testing_class) * 100
      
      # Precision = TP / TP + FP
      precision = (True_Positive / (True_Positive + False_Positive))*100
      
      # Sensitivity = TP / TP + FN
      sensitivity = (True_Positive / (True_Positive + False_Negative))*100

      
      #MCC = TP*TN - FN*FP / sqrt((TP+FN)*(TN+FP)*(TP+FP)*(TN+FN))
      mcc = ((True_Positive * True_Negative) - (False_Positive * False_Negative)) / math.sqrt((True_Positive + False_Negative) * (True_Negative + False_Positive)*(True_Positive + False_Positive)*(True_Negative + False_Negative)) *100
      
      # AUC score
      lr_probs = model.predict_proba(x[test])
      # lr_probs = lr_probs[:, 1]
      lr_auc = (roc_auc_score(y1[test], lr_probs, multi_class="ovr",average="macro"))*100


      print(" Accuracy :%.2f%%   Precision :%.2f%%   Sensitivity :%.2f%%   MCC :%.2f%%   AUC :%.2f%%"% (accuracy,precision,sensitivity,mcc,lr_auc))
      accu_lst.append(accuracy)
      sen_lst.append(sensitivity)
      pre_lst.append(precision)
      acc_lst.append(acc)
      mcc_lst.append(mcc)
      auc_lst.append(lr_auc)
  print ('10 Fold cross-validation performance measurement :')
  print("Accuracy    :%.2f%% (+/- %.2f%%)" % (numpy.mean(accu_lst), numpy.std(accu_lst)))
  print("Precision   :%.2f%% (+/- %.2f%%)" % (numpy.mean(pre_lst), numpy.std(pre_lst)))
  print("Sensitivity :%.2f%% (+/- %.2f%%)" % (numpy.mean(sen_lst), numpy.std(sen_lst)))
  print("MCC         :%.2f%% (+/- %.2f%%)" % (numpy.mean(mcc_lst), numpy.std(mcc_lst)))
  print("AUC         :%.2f%% (+/- %.2f%%)" % (numpy.mean(auc_lst), numpy.std(auc_lst)))
  
  
  
  #independent_test

      
  test_prediction = model.predict(X_test)
  yy_pred=( np.argmax(test_prediction,axis=1))

  results = confusion_matrix(y_test,yy_pred) 
  print ('Independent test set performance measurement :')
  print ('Confusion Matrix :')
  print(results) 
  print ('Accuracy Score :',accuracy_score(y_test,yy_pred))
  print ('Report : ')
  print (classification_report(y_test,yy_pred))

  test_lr_probs = model.predict_proba(X_test)
  test_lr_auc = (roc_auc_score(y_test, test_lr_probs, multi_class="ovr",average="macro"))*100
  print("AUC :%.2f%%"%(test_lr_auc))

  # n_classes=3

  # # Compute ROC curve and ROC area for each class
  # fpr = dict()
  # tpr = dict()
  # test_fpr = dict()
  # test_tpr = dict()
  # roc_auc = dict()
  # test_roc_auc = dict()

  # for i in range(n_classes):
  #     fpr[i], tpr[i], _ = roc_curve(y1[test][:, i], y_pred[:, i])
  #     roc_auc[i] = auc(fpr[i], tpr[i])

  #     test_fpr[i], test_tpr[i], _ = roc_curve(y_test[:, i], yy_pred[:, i])
  #     test_roc_auc[i] = auc(fpr[i], tpr[i])

  # # Plot of a ROC curve for a specific class
  # for i in range(n_classes):
  #     plt.figure()
  #     plt.plot(fpr[i], tpr[i], label='10-fold cross-validation(AUC)= %0.2f' % roc_auc[i])
  #     plt.plot(test_fpr[i], test_tpr[i], label='Independent test(AUC)= %0.2f' % test_roc_auc[i])
  #     plt.plot([0, 1], [0, 1], 'k--')
  #     plt.xlim([0.0, 1.0])
  #     plt.ylim([0.0, 1.05])
  #     plt.xlabel('False Positive Rate')
  #     plt.ylabel('True Positive Rate')
  #     plt.title('Receiver operating characteristic example')
  #     plt.legend(loc="lower right")
  #     plt.show()






 Accuracy :85.71%   Precision :100.00%   Sensitivity :66.67%   MCC :73.03%   AUC :85.42%


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in true_divide


 Accuracy :88.89%   Precision :nan%   Sensitivity :0.00%   MCC :nan%   AUC :89.06%
 Accuracy :93.33%   Precision :100.00%   Sensitivity :83.33%   MCC :86.60%   AUC :100.00%
 Accuracy :69.23%   Precision :66.67%   Sensitivity :40.00%   MCC :31.75%   AUC :84.61%
 Accuracy :85.71%   Precision :100.00%   Sensitivity :60.00%   MCC :70.06%   AUC :92.21%
10 Fold cross-validation performance measurement :
Accuracy    :84.58% (+/- 8.17%)
Precision   :nan% (+/- nan%)
Sensitivity :50.00% (+/- 28.60%)
MCC         :nan% (+/- nan%)
AUC         :90.26% (+/- 5.58%)
Independent test set performance measurement :
Confusion Matrix :
[[ 7  0  0]
 [ 3  2  7]
 [ 0  1 10]]
Accuracy Score : 0.6333333333333333
Report : 
              precision    recall  f1-score   support

           0       0.70      1.00      0.82         7
           1       0.67      0.17      0.27        12
           2       0.59      0.91      0.71        11

    accuracy                           0.63        30
   macro avg       0.65

In [ ]:
|